In [ ]:
pip install opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 19.6 MB/s eta 0:00:00


In [ ]:
pip install SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 21.1 MB/s eta 0:00:00


In [ ]:
pip install scipy

In [ ]:
import os
import torch
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import cv2
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [ ]:
#Any center dataset can be accessed since this is a class where various instances can be created

class center_dataset():
  def __init__(self,folder_path):
    self.image_paths=[]
    self.output_classes=[]

    for categories in os.listdir(folder_path):
      for image_name in os.listdir(os.path.join(folder_path,categories)):
        self.image_paths.append(os.path.join(folder_path,categories,image_name))
        if (categories=="muscle invasive bladder"):
          self.output_classes.append(0)
        else:
          self.output_classes.append(1)

  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self,idx):
    # image=Image.open(self.image_paths[idx])
    # image_label=self.output_classes[idx]
    # print(self.image_paths[idx])
    return self.image_paths[idx],self.output_classes[idx]

In [ ]:
center_1_dataset=center_dataset("/content/drive/MyDrive/Colab Notebooks/Data/Bladder_Cancer_Detection/Center_1")
center_2_dataset=center_dataset("/content/drive/MyDrive/Colab Notebooks/Data/Bladder_Cancer_Detection/Center_2")
center_3_dataset=center_dataset("/content/drive/MyDrive/Colab Notebooks/Data/Bladder_Cancer_Detection/Center_3")
center_4_dataset=center_dataset("/content/drive/MyDrive/Colab Notebooks/Data/Bladder_Cancer_Detection/Center_4")

In [ ]:
import numpy as np
from PIL import Image
import SimpleITK as sitk
from scipy.ndimage import label, find_objects

def bias_field_correction(channel_np):
    sitk_image = sitk.GetImageFromArray(channel_np)
    mask = sitk.OtsuThreshold(sitk_image, 0, 1, 200)
    corrected = sitk.N4BiasFieldCorrection(sitk_image, mask)
    return sitk.GetArrayFromImage(corrected)

def crop_to_white_blob(image_np):
    gray = np.mean(image_np[:, :, :3], axis=2)
    binary = gray > 200
    labeled, num_features = label(binary)
    center_y, center_x = gray.shape[0] // 2, gray.shape[1] // 2
    selected_bbox = None
    objects = find_objects(labeled)

    for i, slice_tuple in enumerate(objects, start=1):
        y_slice, x_slice = slice_tuple
        if (y_slice.start <= center_y < y_slice.stop) and (x_slice.start <= center_x < x_slice.stop):
            selected_bbox = slice_tuple
            break

    if selected_bbox:
        y_slice, x_slice = selected_bbox
        return image_np[y_slice, x_slice, :]
    return image_np  # Return original if no valid crop found

def zscore_normalize(image_np):
    for c in range(image_np.shape[2]):
        channel = image_np[:, :, c]
        mean, std = channel.mean(), channel.std()
        image_np[:, :, c] = (channel - mean) / std if std > 0 else (channel - mean)
    return image_np

def rescale_to_uint8(image_np):
    rescaled = np.zeros_like(image_np, dtype=np.uint8)
    for c in range(image_np.shape[2]):
        channel = image_np[:, :, c]
        min_val, max_val = channel.min(), channel.max()
        if max_val > min_val:
            rescaled[:, :, c] = ((channel - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    return rescaled

def preprocess_image(image_path, store_path):
    image = Image.open(image_path).convert("RGBA")
    image_np = np.array(image, dtype=np.float32)

    corrected = np.zeros_like(image_np)
    for c in range(image_np.shape[2]):
        corrected[:, :, c] = bias_field_correction(image_np[:, :, c])
    image_np = corrected

    image_np = crop_to_white_blob(image_np)

    # Ensure correct shape for PIL conversion
    image_np = np.clip(image_np, 0, 255).astype(np.uint8)

    # Convert to PIL, resize, and convert back to NumPy
    pil_img = Image.fromarray(image_np, mode="RGBA")
    pil_img = pil_img.resize((320, 320), resample=Image.BILINEAR)
    image_np = np.array(pil_img, dtype=np.float32)

    image_np = zscore_normalize(image_np)
    image_uint8 = rescale_to_uint8(image_np)

    # Ensure shape is valid for PIL
    if image_uint8.shape[2] == 4:
        final_img = Image.fromarray(image_uint8, mode="RGBA")
    else:
        final_img = Image.fromarray(image_uint8[:, :, :3], mode="RGB")

    final_img.save(store_path)


In [ ]:
for i in range(center_1_dataset.__len__()):
  image_path,gt=center_1_dataset.__getitem__(i)
  if (gt==0):
    preprocess_image(image_path, "/content/drive/MyDrive/Colab Notebooks/Data/Bladder_Cancer_Detection/Center_1_Preprocessed/muscle invasive bladder/image_"+str(i)+".png")
  else:
    preprocess_image(image_path, "/content/drive/MyDrive/Colab Notebooks/Data/Bladder_Cancer_Detection/Center_1_Preprocessed/non muscle invasive bladder/image_"+str(i)+".png")

In [ ]:
for i in range(center_2_dataset.__len__()):
  image_path,gt=center_2_dataset.__getitem__(i)
  if (gt==0):
    preprocess_image(image_path, "/content/drive/MyDrive/Colab Notebooks/Data/Bladder_Cancer_Detection/Center_2_Preprocessed/muscle invasive bladder/image_"+str(i)+".png")
  else:
    preprocess_image(image_path, "/content/drive/MyDrive/Colab Notebooks/Data/Bladder_Cancer_Detection/Center_2_Preprocessed/non muscle invasive bladder/image_"+str(i)+".png")


In [ ]:
for i in range(center_3_dataset.__len__()):
  image_path,gt=center_3_dataset.__getitem__(i)
  if (gt==0):
    preprocess_image(image_path, "/content/drive/MyDrive/Colab Notebooks/Data/Bladder_Cancer_Detection/Center_3_Preprocessed/muscle invasive bladder/image_"+str(i)+".png")
  else:
    preprocess_image(image_path, "/content/drive/MyDrive/Colab Notebooks/Data/Bladder_Cancer_Detection/Center_3_Preprocessed/non muscle invasive bladder/image_"+str(i)+".png")



In [ ]:
for i in range(center_4_dataset.__len__()):
  image_path,gt=center_4_dataset.__getitem__(i)
  if (gt==0):
    preprocess_image(image_path, "/content/drive/MyDrive/Colab Notebooks/Data/Bladder_Cancer_Detection/Center_4_Preprocessed/muscle invasive bladder/image_"+str(i)+".png")
  else:
    preprocess_image(image_path, "/content/drive/MyDrive/Colab Notebooks/Data/Bladder_Cancer_Detection/Center_4_Preprocessed/non muscle invasive bladder/image_"+str(i)+".png")
